# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [3]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [8]:
path = '../WeatherPy/weather_output.csv'
df = pd.read_csv(path, encoding = 'utf-8')
df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Date,Wind Speed,Country
0,mar del plata,-38.0023,-57.5575,285.15,87,40,1620739065,3.60,AR
1,bud,39.4470,-86.1758,281.48,73,100,1620739260,4.02,US
2,vaini,-21.2000,-175.2000,297.15,78,20,1620739260,4.63,TO
3,tilichiki,60.4667,166.1000,266.13,92,85,1620739260,3.57,RU
4,camacari,-12.6975,-38.3242,301.15,51,20,1620739260,5.66,BR
...,...,...,...,...,...,...,...,...,...
550,olga,43.7444,135.2892,276.70,96,24,1620739938,0.25,RU
551,shingu,33.7333,135.9833,289.73,78,100,1620739938,1.41,JP
552,maumere,-8.6199,122.2111,298.42,71,2,1620739999,2.00,ID
553,kalabo,-14.9700,22.6814,301.39,20,4,1620739999,3.17,ZM


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [ ]:
#fig = gmap.figure(...)
#once fig is added, then don't need to do it again. Using the same figure, add layers onto for the rest.

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [ ]:
# NOTE: Do not change any of the code in this cell
#Gretel#Don't need to change code as long as nmae is consistent
#look at google gmaps documentation
# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map


# Display figure
